In [1]:
import pymol
from pymol import cmd
pymol.finish_launching()

cmd.load("./target_pdbs/1und.pdb", "molecule")
cmd.h_add("molecule")  # Adding hydrogen

molecule = "molecule"
chain = "A"            # Replace with your actual chain ID
n_term_resi = "1"      # Replace with your N-terminal residue number
c_term_resi = "36"    # Replace with your C-terminal residue number

# Capping N-terminus
cmd.wizard("mutagenesis")
cmd.get_wizard().set_n_cap("acet")
selection = f"/{molecule}//{chain}/{n_term_resi}"
cmd.get_wizard().do_select(selection)
cmd.get_wizard().apply()

# Capping C-terminus
cmd.get_wizard().set_c_cap("nmet")
selection = f"/{molecule}//{chain}/{c_term_resi}"
cmd.get_wizard().do_select(selection)
cmd.get_wizard().apply()

cmd.set_wizard()


 PyMOL(TM) Molecular Graphics System, Version 3.0.0.
 Copyright (c) Schrodinger, LLC.
 All Rights Reserved.
 
    Created by Warren L. DeLano, Ph.D. 
 
    PyMOL is user-supported open-source software.  Although some versions
    are freely available, PyMOL is not in the public domain.
 
    If PyMOL is helpful in your work or study, then please volunteer 
    support for our ongoing efforts to create open and affordable scientific
    software by purchasing a PyMOL Maintenance and/or Support subscription.

    More information can be found at "http://www.pymol.org".
 
    Enter "help" for a list of commands.
    Enter "help <command-name>" for information on a specific command.

 Hit ESC anytime to toggle between text and graphics.

Selected!
 ExecutiveRMSPairs: RMSD =    0.416 (4 to 4 atoms)
 Mutagenesis: 4 rotamers loaded.
 Rotamer 1/4, strain=16.77
Selected!
 ExecutiveRMSPairs: RMSD =    0.618 (4 to 4 atoms)
 Mutagenesis: no phi/psi, using backbone-independent rotamers.
 Mutagenesi

In [ ]:
import os
import sys
sys.path.append('.')

# Import the Preprocess class and arguments module
from src.AIMD.preprocess import Preprocess
from src.AIMD import arguments

# Set the required parameters for arguments.init
argv = [
    "--prot-file", "target_pdbs/processed_1und_amber_hydr.pdb",
    "--log-dir", "saved_logs",
    "--temp-k", "300",
    "--max-cyc", "10",
    "--preprocess-method", "FF19SB",
    "--seed", "12345"
]

# Initialize arguments with the provided parameters
arguments.init(argv)

# Instantiate the Preprocess class using the initialized arguments
preprocessor = Preprocess(
    prot_path=arguments.get().prot_file[:-4],  # Remove .pdb extension
    utils_dir="src/utils",
    command_save_path=arguments.get().log_dir,
    preprocess_method=arguments.get().preprocess_method,
    log_dir=arguments.get().log_dir,
    temp_k=arguments.get().temp_k
)

# Run the preprocessing (relaxation)
preeq_pdb, preeq_nowat_pdb = preprocessor.run_preprocess()



In [1]:
import os

import torch.multiprocessing as mp

from src.AIMD import arguments, envflags
from src.AIMD.preprocess import Preprocess
from src.AIMD.protein import Protein
from src.AIMD.simulator import SolventSimulator, NoSolventSimulator
from src.Calculators.calculator import patch_check_state
from src.utils.pdb import fix_atomic_numbers, read_protein
from src.utils.system import redir_output

patch_check_state()


prot = Protein(read_protein(preeq_pdb), pdb4params=preeq_nowat_pdb)
fix_atomic_numbers(preeq_pdb, prot)


args = arguments.init()

simulator_class = SolventSimulator if args.solvent else NoSolventSimulator
simulator = simulator_class(
    prot=prot,
    log_path=args.log_dir,
    preeq_steps=args.preeq_steps,
    temp_k=args.temp_k,
    utils_dir=args.utils_dir,
    pdb_file=preeq_pdb,
    nowat_pdb_file=preeq_nowat_pdb,
    mmcalc_type=args.mm_method,
    preprocess_method=args.preprocess_method,
    dev_strategy=args.device_strategy,
)

simulator.set_calculator(
    ckpt_path=args.ckpt_path,
    ckpt_type=args.ckpt_type,
    nbcalc_type=args.fragment_longrange_calc,
)

simulator.simulate(
    prot_name=args.prot_name,
    simulation_steps=args.sim_steps,
    time_step=args.timestep,
    record_per_steps=args.record_per_steps,
    hydrogen_constraints=args.constraints,
    seed=args.seed,
    restart=args.restart,
    build_frames=args.build_frames,
)

NameError: name 'preeq_pdb' is not defined